LSTM with Attention 을 이용한 뉴스 데이터 요약

https://velog.io/@cardy20/LSTM-with-Attention-%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EB%89%B4%EC%8A%A4-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EC%9A%94%EC%95%BD

In [1]:
from google.colab import drive
drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/new_project/project2/'
traindata_path = '/content/drive/MyDrive/new_project/project2/traindata/문서요약 텍스트/'
rawdata_path = '/content/drive/MyDrive/new_project/project2/rawdata/'

Mounted at /content/drive


In [2]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 32.6 MB/s eta 0:00:00


In [3]:
import re
import json
import pandas as pd

import os
import time
import numpy as np
from tqdm import tqdm
# 시각화
import matplotlib.pyplot  as plt
# 형태소 분석기
from konlpy.tag import Okt

from collections import Counter
from tqdm.auto import tqdm
import pickle as pkl


In [24]:
data = pd.read_csv(f"{traindata_path}/train_dataset.csv", nrows = 1000)
print('전체 리뷰 개수 :',(len(data)))

전체 리뷰 개수 : 1000


In [25]:
data.head()

,id,category,media_type,media_sub_type,media_name,char_count,publish_date,title,abstractive,text_all,text_sentence,score_read,score_acc,score_info,score_trust,extractive2,dataset
0,290741778,종합,online,지역지,광양신문,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요",전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,ha당 조사료 400만원…작물별 차등 지원 전라남도가 쌀 과잉문제를 근본적으로 해결...,"['ha당 조사료 400만원…작물별 차등 지원', '전라남도가 쌀 과잉문제를 근본적...",4,3,3,3,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,train
1,290741792,종합,online,지역지,광양신문,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 여수시는 원도심 일대에서 추진된 컬러...","['8억 투입, 고소천사벽화·자산마을에 색채 입혀', '여수시는 원도심 일대에서 추...",4,4,5,4,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,train
2,290741793,스포츠,online,지역지,광양신문,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,전남드래곤즈 해맞이 다짐…선수 영입 활발 전남드래곤즈(사장 신승재)는 지난 4일 구...,"['전남드래곤즈 해맞이 다짐…선수 영입 활발', '전남드래곤즈(사장 신승재)는 지난...",4,4,4,4,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,train
3,290741794,정치,online,지역지,광양신문,746,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작...","11~24일, 매실·감·참다래 등 지역특화작목 광양시는 오는 11일부터 24일까지 ...","['11~24일, 매실·감·참다래 등 지역특화작목', '광양시는 오는 11일부터 2...",4,4,5,4,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,train
4,290741797,종합,online,지역지,광양신문,978,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 타이완의 크루즈관광객 4000...","['홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항', '타이완의 크루즈관광객...",4,4,4,4,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,train


In [32]:
df = data[['media_name', 'id', 'text_all','abstractive']]
df.columns = ['media', 'id', 'text', 'abs']
df.head()

,media,id,text,abs
0,광양신문,290741778,ha당 조사료 400만원…작물별 차등 지원 전라남도가 쌀 과잉문제를 근본적으로 해결...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,광양신문,290741792,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 여수시는 원도심 일대에서 추진된 컬러...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,광양신문,290741793,전남드래곤즈 해맞이 다짐…선수 영입 활발 전남드래곤즈(사장 신승재)는 지난 4일 구...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,광양신문,290741794,"11~24일, 매실·감·참다래 등 지역특화작목 광양시는 오는 11일부터 24일까지 ...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,광양신문,290741797,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 타이완의 크루즈관광객 4000...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [33]:
df.dropna(inplace = True)

<ipython-input-33-d1ada024d2eb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace = True)


In [34]:
# 전처리 - 한글만 남기기
# f = lambda x : ' '.join(x)
# df['text'] = df['text'].apply(f)

class HangulExtractor:
    def __init__(self):
        self.pattern = re.compile('[^ ㄱ-ㅣ가-힣]+')

    def __call__(self, sentence):
        return self.pattern.sub('', sentence)

he = HangulExtractor()

df['text'] = df['text'].apply(he)
df['abs'] = df['abs'].apply(he)

<ipython-input-34-887ddde2b2d4>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(he)
<ipython-input-34-887ddde2b2d4>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['abs'] = df['abs'].apply(he)


In [36]:
df.head()

,media,id,text,abs
0,광양신문,290741778,당 조사료 만원작물별 차등 지원 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 ...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,광양신문,290741792,억 투입 고소천사벽화자산마을에 색채 입혀 여수시는 원도심 일대에서 추진된 컬러빌리지...,여수시는 컬러빌리지 사업에 억원을 투입하여 색채와 빛 도시를 완성하여 고소천사벽화마...
2,광양신문,290741793,전남드래곤즈 해맞이 다짐선수 영입 활발 전남드래곤즈사장 신승재는 지난 일 구봉산 해...,전남드래곤즈 임직원과 선수단이 일 구봉산 정상에 올라 일출을 보며 년 구단 목표 달...
3,광양신문,290741794,일 매실감참다래 등 지역특화작목 광양시는 오는 일부터 일까지 농업인교육관과 읍면동 ...,광양시는 농업인들의 경쟁력을 높이고 소득안정을 위해 매실감참다래 등 지역특화작목 중...
4,광양신문,290741797,홍콩 크루즈선사아쿠아리우스 월 여수항 입항 타이완의 크루즈관광객 여명이 올해 두 ...,올해 월과 월 두 차례에 걸쳐 타이완의 크루즈 관광객 여명이 여수에 입항해 전남의 ...


In [40]:
###요약 태스크에 대한 간단한 소개.
# article original을 받아 abstractive를 생성해내는 것이 목표.
# 디코더는 아래와 같은 형태가 인코딩 되어 들어 갈 것.

df['decoder_input'] = df['abs'].apply(lambda x : '[BOS] '+ x)
df['decoder_target'] = df['abs'].apply(lambda x : x + ' [EOS]')
df.head()

<ipython-input-40-ddc0e602455c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decoder_input'] = df['abs'].apply(lambda x : '[BOS] '+ x)
<ipython-input-40-ddc0e602455c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['decoder_target'] = df['abs'].apply(lambda x : x + ' [EOS]')


,media,id,text,abs,decoder_input,decoder_target
0,광양신문,290741778,당 조사료 만원작물별 차등 지원 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 ...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...,[BOS] 전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 ...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,광양신문,290741792,억 투입 고소천사벽화자산마을에 색채 입혀 여수시는 원도심 일대에서 추진된 컬러빌리지...,여수시는 컬러빌리지 사업에 억원을 투입하여 색채와 빛 도시를 완성하여 고소천사벽화마...,[BOS] 여수시는 컬러빌리지 사업에 억원을 투입하여 색채와 빛 도시를 완성하여 고...,여수시는 컬러빌리지 사업에 억원을 투입하여 색채와 빛 도시를 완성하여 고소천사벽화마...
2,광양신문,290741793,전남드래곤즈 해맞이 다짐선수 영입 활발 전남드래곤즈사장 신승재는 지난 일 구봉산 해...,전남드래곤즈 임직원과 선수단이 일 구봉산 정상에 올라 일출을 보며 년 구단 목표 달...,[BOS] 전남드래곤즈 임직원과 선수단이 일 구봉산 정상에 올라 일출을 보며 년 구...,전남드래곤즈 임직원과 선수단이 일 구봉산 정상에 올라 일출을 보며 년 구단 목표 달...
3,광양신문,290741794,일 매실감참다래 등 지역특화작목 광양시는 오는 일부터 일까지 농업인교육관과 읍면동 ...,광양시는 농업인들의 경쟁력을 높이고 소득안정을 위해 매실감참다래 등 지역특화작목 중...,[BOS] 광양시는 농업인들의 경쟁력을 높이고 소득안정을 위해 매실감참다래 등 지역...,광양시는 농업인들의 경쟁력을 높이고 소득안정을 위해 매실감참다래 등 지역특화작목 중...
4,광양신문,290741797,홍콩 크루즈선사아쿠아리우스 월 여수항 입항 타이완의 크루즈관광객 여명이 올해 두 ...,올해 월과 월 두 차례에 걸쳐 타이완의 크루즈 관광객 여명이 여수에 입항해 전남의 ...,[BOS] 올해 월과 월 두 차례에 걸쳐 타이완의 크루즈 관광객 여명이 여수에 입항...,올해 월과 월 두 차례에 걸쳐 타이완의 크루즈 관광객 여명이 여수에 입항해 전남의 ...


In [38]:
text_len = [len(s.split()) for s in df['text']]
summary_len = [len(s.split()) for s in df['abs']]

print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
print('요약의 최소 길이 : {}'.format(np.min(summary_len)))
print('요약의 최대 길이 : {}'.format(np.max(summary_len)))
print('요약의 평균 길이 : {}'.format(np.mean(summary_len)))

텍스트의 최소 길이 : 113
텍스트의 최대 길이 : 435
텍스트의 평균 길이 : 216.207
요약의 최소 길이 : 7
요약의 최대 길이 : 50
요약의 평균 길이 : 21.316


In [12]:
# 고정된 시퀀스길이 설정.
# Vocab 생성
# Vocab 형태 확인.
# Vocab 제공.

In [13]:
#### tokenizer 저장

# from collections import Counter
# from tqdm.auto import tqdm
# import pickle as pkl

# def get_vocab(split_dataset):
#     lines = []
#     for line in tqdm(split_dataset):
#         lines.extend(line)

#     counter = Counter(lines)
#     voca_freq = dict(counter)
#     vocab = {'[PAD]': 0,
#              '[BOS]': 1,
#              '[EOS]': 2,
#              '[UNK]': 3}
#     for word, freq in voca_freq.items():
#         if freq <= 5:
#             continue
#         vocab[word] = len(vocab)
#     return vocab
# # vocab 생성
# art_split = []
# abs_split = []

# tokenizer = Okt()
# for s in tqdm(df['text']):
#     art_split.append(tokenizer.morphs(s))
# for s in tqdm(df['abs']):
#     abs_split.append(tokenizer.morphs(s))


# src_vocab = get_vocab(art_split)
# trg_vocab = get_vocab(abs_split)


# # vocab 저장
# with open('/content/drive/MyDrive/new_project/project2/vocab/src_vocab.pkl', 'wb') as fw:
#     pkl.dump(src_vocab, fw)
# with open('/content/drive/MyDrive/new_project/project2/vocab/trg_vocab.pkl', 'wb') as fw:
#     pkl.dump(trg_vocab, fw)

In [39]:
# load the vocabulary

with open(f'{base_path}vocab/src_vocab.pkl', 'rb') as fr:
    src_vocab = pkl.load(fr)
with open(f'{base_path}vocab/trg_vocab.pkl', 'rb') as fr:
    trg_vocab = pkl.load(fr)

In [41]:
def word2index(split_data, src_vocab, trg_vocab):
    srcs=[]
    trgs=[]
    start_time = time.time()
    for src, trg in tqdm(zip(split_data['text'], split_data['abs']), total=len(split_data['text'])):
        src_ind = [src_vocab[tok] if tok in src_vocab else src_vocab['[UNK]'] for tok in src]
        if len(src_ind) < SRC_MAX_SEQ:
            src_ind += [src_vocab['[PAD]']] * (SRC_MAX_SEQ - len(src_ind))
        else:
            src_ind = src_ind[:SRC_MAX_SEQ]
        srcs.append(src_ind)

        trg_ind = [trg_vocab['[BOS]']] + [trg_vocab[tok] if tok in trg_vocab else trg_vocab['[UNK]'] for tok in trg] + [trg_vocab['[EOS]']]
        if len(trg_ind) < TRG_MAX_SEQ:
            trg_ind += [trg_vocab['[PAD]']] * (TRG_MAX_SEQ - len(trg_ind))
        else:
            trg_ind = trg_ind[:TRG_MAX_SEQ-1] + [trg_vocab['[EOS]']]
        trgs.append(trg_ind)
    return srcs, trgs

In [42]:
# 무작위로 섞기
indices = np.arange(df['text'].shape[0])
np.random.shuffle(indices)

df = df.iloc[indices]

In [43]:
n_of_val = int(len(df['text'])*0.2)
print('테스트 데이터의 수 :',n_of_val)

테스트 데이터의 수 : 200


In [44]:
train_split_data = df.iloc[:-n_of_val]
val_split_data = df.iloc[-n_of_val:]

In [45]:
len(train_split_data)

800

In [46]:
len(val_split_data)

200

In [56]:
SRC_MAX_SEQ = 700
TRG_MAX_SEQ = 100

In [57]:
train_srcs, train_trgs = word2index(train_split_data, src_vocab, trg_vocab)
val_srcs, val_trgs = word2index(val_split_data, src_vocab, trg_vocab)

  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

In [58]:
print(len(train_srcs))
print(len(train_trgs))

print(len(val_srcs))
print(len(val_trgs))

800
800
200
200


In [59]:
print(len(train_srcs[0]))
print(len(train_trgs[0]))

print(len(val_srcs[0]))
print(len(val_trgs[0]))

700
100
700
100


In [60]:
encoder_input_train = train_srcs
encoder_input_val = val_srcs

# decoder_input_train = train_trgs[:, :TRG_MAX_SEQ-1]
# decoder_input_val = val_trgs[:, :TRG_MAX_SEQ-1]

# decoder_target_train = train_trgs[:, 1: TRG_MAX_SEQ]
# decoder_target_val = val_trgs[:, 1: TRG_MAX_SEQ]
# print("ENCODER")
# print("  input_train : ", encoder_input_train.shape)
# print("  input_val   : ", encoder_input_val.shape)

# print("\nDECODER")
# print("  input_train : ", decoder_input_train.shape)
# print("  input_val   : ", decoder_input_val.shape)
# print("  target_train: ", decoder_target_train.shape)
# print("  target_val  : ", decoder_target_val.shape)

In [70]:
decoder_input_train = np.array([])
decoder_target_train = np.array([])

for trg in train_trgs:
    decoder_input_train.append(trg[:-1])
    decoder_target_train.append(trg[1:])



AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [71]:
decoder_input_val = np.array([])
decoder_target_val = np.array([])

for trg in val_trgs:
    decoder_input_val.append(trg[:-1])
    decoder_target_val.append(trg[1:])


In [63]:
print(len(decoder_input_train))
print(len(decoder_input_train[500]))

800
99


In [72]:
decoder_target_val[100]

3.0

In [67]:
import datetime
import gensim
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 미리 만들어진 어텐션 레이어 가져오기.
import urllib
urllib.request.urlretrieve("https://raw.githubusercontent.com/thushv89/attention_keras/master/src/layers/attention.py",\
                           filename="attention.py")
from attention import AttentionLayer

# 하이퍼 파라미터 정의
SRC_MAX_SEQ = 700
TRG_MAX_SEQ = 50
EMB_DIM = 200
HIDDEN_SIZE = 256
BATCH_SIZE = 256
EPOCHS = 30


# 모델 정의

# 인코더
encoder_inputs = Input(shape=(SRC_MAX_SEQ, ))
enc_emb = Embedding(len(src_vocab), EMB_DIM)(encoder_inputs)

encoder_lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True ,dropout = 0.4, recurrent_dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

encoder_lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True, dropout=0.4, recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(HIDDEN_SIZE, return_state=True, return_sequences=True, dropout=0.4, recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# 디코더
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(len(trg_vocab), EMB_DIM)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(HIDDEN_SIZE, return_sequences = True, return_state = True, dropout = 0.4, recurrent_dropout=0.2)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = [state_h, state_c])

# 어텐션 함수 적용.
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# 어텐션의 결과와 디코더의 hidden state들을 연결.
decoder_concat_input = Concatenate(axis = -1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층.
decoder_softmax_layer = Dense(len(trg_vocab), activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 호출.
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 700)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 700, 200)             354000    ['input_1[0][0]']             
                                                                                                  
 lstm (LSTM)                 [(None, 700, 256),           467968    ['embedding[0][0]']           
                              (None, 256),                                                        
                              (None, 256)]                                                        
                                                                                              

In [68]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)
# tb_callback = tf.keras.callbacks.TensorBoard(f'{base_path}vocab', update_freq=1)



In [69]:

history = model.fit(x = [encoder_input_train + decoder_input_train],
                    y = decoder_target_train,
                    validation_data = ([encoder_input_val, decoder_input_val], decoder_target_val),
                    batch_size = BATCH_SIZE,
                    callbacks= [es_callback],
                    epochs = 3)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (1, 1600) + inhomogeneous part.